In [ ]:
import torchvision.models as models
from torchvision.models import DenseNet121_Weights
model = models.densenet121(weights=DenseNet121_Weights.DEFAULT).eval()

In [ ]:
onnxmodel = torch.onnx.export(model,
                  (img,),
                  'first.onnx',
                  dynamic_shapes = {'x': {0: "batch_size"}},
                  input_names = ['x'],
                  output_names = ['logits'],
                  dynamo = True          
                  )

In [ ]:
import onnx
softmax_node = onnx.helper.make_node('Softmax',
                  inputs = ['logits'],
                    outputs = ['soft_output'])
argmax_node = onnx.helper.make_node('ArgMax',
    inputs = ['soft_output'],
    outputs = ['final_out'],
    axis = 1,
    keepdims = 0)

In [ ]:
model = onnx.load('/content/first.onnx')
model.graph.node.extend([softmax_node, argmax_node])

model.graph.output.extend([
    onnx.helper.make_tensor_value_info('soft_output',onnx.TensorProto.FLOAT, [None, None]),
    onnx.helper.make_tensor_value_info('final_out', onnx.TensorProto.INT64, [None])
])

del model.graph.output[0]
del model.graph.output[0]

onnx.save_model(model, 'modified.onnx')

In [ ]:
import time
import onnxruntime as rt
import numpy as np
sess_options = rt.SessionOptions()

sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL

sess = rt.InferenceSession("/content/modified.onnx", sess_options = sess_options, providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'])
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
io_binding = sess.io_binding()

In [ ]:

io_binding.bind_input(
    name = input_name,
    device_type = 'cuda',
    device_id = 0,
    element_type = np.float32,
    shape = tuple(i.shape),
    buffer_ptr = i.data_ptr(),
    )


io_binding.bind_output(name = output_name, device_type= 'cpu')

sess.run_with_iobinding(io_binding)
Y = io_binding.copy_outputs_to_cpu()[0]